# Sequence Models
This tutorial follows the [Sequence Models and Long-Short Term Memory Networks](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html) tutorial on the Pytorch webpage. The original author is Robert Guthrie.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

## Introduction
Here is a briefly example showing how LSTMs work in Pytorch.

In [2]:
lstm = nn.LSTM(3, 3)  # Input dimension is 3, output dimension is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # Make a sequence of length 5

# Initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence on element at a time.
    # After each time step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    
# Alternate way that doesn't use a for loop:
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # Clean out hidden state
out, hidden = lstm(inputs, hidden)
print('out: {}'.format(out))
print('hidden: {}'.format(hidden))

out: tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]])
hidden: (tensor([[[-0.3368,  0.0959, -0.0538]]]), tensor([[[-0.9825,  0.4715, -0.0633]]]))


## Example: An LSTM for Part-of-Speech Tagging
In this example, a model will be created that can predict the part-of-speech for each word in a sentence.

### Prepare the data:

In [3]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

training_data = [
    ("The dog ate the apple.".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book.".split(), ["NN", "V", "DET", "NN"])
]
training_sentences = [training_data[x][0] for x in range(len(training_data))]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print('word_to_ix: {}'.format(word_to_ix))
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}
ix_to_tag = {0: "DET", 1: "NN", 2: "V"}

word_to_ix: {'the': 3, 'read': 6, 'apple.': 4, 'ate': 2, 'that': 7, 'The': 0, 'dog': 1, 'book.': 8, 'Everybody': 5}


### Set Hyperparameters

In [4]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
LEARNING_RATE = 0.1
NUM_EPOCHS = 300

### Create the model

In [5]:
class LSTMTagger(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM: Inputs are embeddings, outputs are hidden states
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        # Linear layer maps hidden space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        # Initialize the hidden state. The axes correspond to 
        # (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

### Train the model:

In [6]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

# Print out the scores before the model has been trained:
store_initial_scores = []
with torch.no_grad():
    for sentence in training_sentences:
        inputs = prepare_sequence(sentence, word_to_ix)
        tag_scores = model(inputs)
        store_initial_scores.append(tag_scores)
    
for epoch in range(NUM_EPOCHS):
    for sentence, tags in training_data:
        # Set gradients equal to zero after each intance
        model.zero_grad()
        
        # Initialize hidden state of LSTM after each intance
        model.hidden = model.init_hidden()
        
        # Turn inputs into tensors of word indices
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        # Run forward pass
        tag_scores = model(sentence_in)
        
        # Compute the loss, gradients, and update the parameters
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        
# Print out the scores after training the model
store_initial_scores.reverse()
with torch.no_grad():
    for sentence in training_sentences:
        inputs = prepare_sequence(sentence, word_to_ix)
        tag_scores = model(inputs)

        print('Original sentence: {}'.format(' '.join(sentence)))
        print('Initial scores: {}'.format(store_initial_scores.pop()))
        print('Final scores: {}'.format(tag_scores))
        max_values, max_indices = torch.max(tag_scores, 1)
        predictions = [ix_to_tag[x] for x in max_indices.numpy()]
        print('Predicted part-of-speech: {}'.format(predictions))
        print('')

Original sentence: The dog ate the apple.
Initial scores: tensor([[-1.1389, -1.2024, -0.9693],
        [-1.1065, -1.2200, -0.9834],
        [-1.1286, -1.2093, -0.9726],
        [-1.1190, -1.1960, -0.9916],
        [-1.0137, -1.2642, -1.0366]])
Final scores: tensor([[-0.0858, -2.9355, -3.5374],
        [-5.2313, -0.0234, -4.0314],
        [-3.9098, -4.1279, -0.0368],
        [-0.0187, -4.7809, -4.5960],
        [-5.8170, -0.0183, -4.1879]])
Predicted part-of-speech: ['DET', 'NN', 'V', 'DET', 'NN']

Original sentence: Everybody read that book.
Initial scores: tensor([[-1.0354, -1.2419, -1.0326],
        [-0.9859, -1.2891, -1.0459],
        [-1.0365, -1.2354, -1.0368],
        [-0.9544, -1.2734, -1.0933]])
Final scores: tensor([[-5.4770, -0.0201, -4.1539],
        [-3.9614, -3.7916, -0.0425],
        [-0.0337, -4.7335, -3.7163],
        [-4.8229, -0.0176, -4.6717]])
Predicted part-of-speech: ['NN', 'V', 'DET', 'NN']

